In [1]:
####################################################################
# VERSION 06 - 2021/04/13
####################################################################
# most used libs
from math import *
import numpy as np
import scipy as sc

####################################################################
# Used if running this python code outside MOOC test platform
if not 'OfflineMode' in globals().keys():
  OfflineMode = not 'anonymous_student_id' in globals().keys()
if OfflineMode: 
  anonymous_student_id = 'student'

####################################################################
# print output if running offline
def OfflinePrint( outtxt='' ):
  if OfflineMode: print( outtxt )

####################################################################
# init numpy random generator
def InitRandomGenerator( problem_id = '' ):
    maxint = np.iinfo(np.int32).max
    rnd_seed = hash( str( anonymous_student_id ) \
                   + str(problem_id) ) % maxint
    np.random.seed( rnd_seed )

####################################################################
def Disturb( val, dig, dist ):
  ss = +1.0 if np.random.uniform() >= 0.5 else -1.0
  rr = np.random.uniform() * ( dist - 0.1 )
  val = val + ss * val * ( rr + 0.1 )
  return round( val, dig ) 

####################################################################
# Uses the random number generator of numpy
# This function should be used instead of random.choice()
def RandomPick( lst ):
  return lst[ np.random.randint( 0, len( lst ) ) ]

####################################################################
# Creates num_entries-1 invalid answers such that
#
#  This function generates the global variables associated with
#  $a_answer_0,..., $a_answer_3 and
#  $a_value_0,..., $a_value_3 and
#
#  <multiplechoiceresponse>
#    <choicegroup type="MultipleChoice">
#      <choice correct="$a_answer_0" > $a_value_0 </choice>
#      <choice correct="$a_answer_1" > $a_value_1 </choice>
#      <choice correct="$a_answer_2" > $a_value_2 </choice>
#      <choice correct="$a_answer_3" > $a_value_3 </choice>
#    </choicegroup>
#  </multiplechoiceresponse>
#
# prefix - the '*' used in the variables *_answer_0 and *_value_0
# value - correct answer
# ndigs - digits after the comma
# suffix - usually the units of values
# entries - total number of displayed values
# minDelta - absolute value of the minimum distance between values
#            The maximum distance between values is 2*minDelta
# limits - 'greater', 'less', 'both'
#
def NumericMultipleChoice( prefix, value, ndigs, suffix, entries, minDelta, 
                             limits = 'both' ):

    def fmt_val( val, dig ):
        return "{vl:.{dg}f}".format( vl = val, dg = dig )

    num_err = entries-1
    options = [ [ "true", value ] ]
    # returns a sign function as a function of the type of limits
    signs = { 'greater': lambda : +1, 
              'less'   : lambda : -1,
              'both'   : lambda : +1 if np.random.uniform() > 0.5 else -1.0 }
    # insert in the options list as a function of the sign
    insertion = { +1 : lambda d : options.append( [ "false", options[-1][1] + d ] ),
                  -1 : lambda d : options.insert( 0, [ "false", options[0][1] - d ]  ) }

    for i in range( num_err ):
        # minDelta <= dd <= 2*minDelta
        dd = np.fabs( minDelta ) * ( 1.0 + np.random.uniform() )
        ss = signs[ limits ]()
        insertion[ ss ]( dd )

    np.random.shuffle(options)
    
    # set the global variables
    gdic = globals()
    for i, opt in enumerate( options ):
        val = fmt_val( opt[1], ndigs ) + ' ' + suffix
        gdic[ '%s_answer_%i' % (prefix,i) ] = opt[0]
        gdic[ '%s_value_%i'  % (prefix,i) ] = val
        OfflinePrint( '[%s_answer_%i] (%-5s) %s' % ( prefix, i, opt[0], val ) )
    OfflinePrint()

####################################################################
# Shuffles all the answers and generates the global variables 
# associated with
#  $a_answer_0,..., $a_answer_3 and
#  $a_value_0,..., $a_value_3 and
#
#  <multiplechoiceresponse>
#    <choicegroup type="MultipleChoice">
#      <choice correct="$a_answer_0" > $a_value_0 </choice>
#      <choice correct="$a_answer_1" > $a_value_1 </choice>
#      <choice correct="$a_answer_2" > $a_value_2 </choice>
#      <choice correct="$a_answer_3" > $a_value_3 </choice>
#    </choicegroup>
#  </multiplechoiceresponse>
#
#-------------------------------------------------------------------
# Parameters:
#
# prefix - the '*' used in the variables *_answer_0 and *_value_0
# correct: string with the correct answer   
# wrong_lst: list of string with wrong answers
#
def StringMultipleChoice( prefix, shuffle_lst, fixed_lst=None ):

    # shuffle and pick num_err entries
    options = list( shuffle_lst ) 
    np.random.shuffle( options )
    if fixed_lst != None:
      options += list( fixed_lst )
    
    # set the global variables
    gdic = globals()
    for i, opt in enumerate( options ):
        gdic[ '%s_answer_%i' % (prefix,i)  ] = opt[0]
        gdic[ '%s_value_%i'  % (prefix,i)  ] = opt[1]
        OfflinePrint( '[%s_answer_%i] (%-5s) %s\n' % ( prefix, i, opt[0], opt[1] ) )
    OfflinePrint()

####################################################################

In [3]:
#####  
D  = RandomPick(( 45., 40., 44., 42., 50., 47., 46., 48.) )  
U  = RandomPick(( 8.0,8.2,8.5,8.4,8.6,8.7,8.8,9.0) )  
theta  = RandomPick(( 6.2, 6.0, 6.09, 5.9, 5.8, 6.4) )  
cLscDmax  = RandomPick(( 63., 61.03, 64., 59., 60., 62.) )  
rho  = RandomPick(( 1.2, 1.15, 1.25) )  
cL  = RandomPick(( 0.92, 0.941, 0.96, 0.98) )  
#####  
#D=40.0
R=D/2.
lbd=6.
#U=8.5
#cLscDmax=61.03
#theta=6.09
#cL=0.941
#rho=1.2
Z=3.
gamma=np.arctan(1./cLscDmax)
gamma_deg=gamma*180./np.pi
# a ##########################
rsR_a=0.6667
Omega=U*lbd/R
x_a=lbd*rsR_a
betam=9.36
epsilon=90.+theta-betam
#print ("a) x_a ",x_a, " ", " betam ",betam, " "," epsilon ",epsilon,  " ")
###########################  
# b ##########################
rsR_b=0.5
x_b=lbd*rsR_b
fc=0.275
c=2*np.pi*U*fc/(Z*Omega*cL)
#print ("b) x_b ",x_b, " ", " fc ",fc, " "," c (m) ", c, " "," epsilon ",epsilon, " ")
###########################  
# c ##########################
cP=0.576
T=cP*0.5*rho*U**3*np.pi*R**2/Omega/1000.
T=round(T,1)
#print ("c) T (kNm) ",T, " ", " fc ",fc, " "," c (m) ", c, " "," epsilon ",epsilon, " ")
###########################  
# d ##########################
a=0.331
aprime=0.0240
Va=(1-a)*U
V2t=-aprime*2*Omega*rsR_b*R
W1=(Va**2+(Omega*rsR_b*R)**2)**0.5
W2=(Va**2+(Omega*rsR_b*R-V2t)**2)**0.5
p1mp2=0.5*rho*(W2**2-W1**2)
p1mp2=round(p1mp2,2)
#print ("d) p1mp2 (Pa) ",p1mp2, " ", " Va(m/s) ",Va, " "," V2t (m/s) ", V2t, " "," W1(m/s) ",W1, " "," W2(m/s) ",W2)
###########################
# # d2 ##########################
betamd=np.arctan(Va/(Omega*rsR_b*R-V2t/2.))
betamd_deg=betamd*180./np.pi
t=2*np.pi*rsR_b*R/Z
p1mp2b=rho*cL*c/t*Va**2/2*np.cos(betamd)/(np.sin(betamd)**2)
p1mp2b=round(p1mp2b,2)
#print ("d) p1mp2 (Pa) ",p1mp2b, " ", " Va(m/s) ",Va, " "," V2t (m/s) ", V2t, " "," W1(m/s) ",W1, " "," betamd_deg ",betamd_deg)
###########################  
# # e ##########################

p1mp2e=rho*cL*c/t*Va**2/2*np.cos(betamd-gamma)/((np.sin(betamd)**2)*np.cos(gamma))
p1mp2e=round(p1mp2e,2)
#print ("e) p1mp2e (Pa) ",p1mp2e, " ", " Va(m/s) ",Va, " "," V2t (m/s) ", V2t, " "," gamma_deg ",gamma_deg, " "," betamd_deg ",betamd_deg)
###########################  
# # # f ##########################
a=1./3.
Fx=round(rho*U**2*(1.-a)*(-2*a)*np.pi*R**2/1000.,1)
#print ("f) Fx (kN) ",Fx, " ", " Va(m/s) ",Va, " "," V2t (m/s) ", V2t, " "," gamma_deg ",gamma_deg, " "," betamd_deg ",betamd_deg)
###########################  

   
   
# prefix - the '*' used in the variables *_answer_0 and *_value_0
# value - correct answer
# ndigs - digits after the comma
# suffix - usually the units of values
# entries - total number of displayed values
# delta - absolute value of the desired mean interval between values
# limits - 'greater', 'smaller', 'both'
#
# def NumericMultipleChoice( prefix, value, ndigs, suffix, entries, gdelta,
#                             limits = 'both' )


nr=" no answer"    
NumericMultipleChoice( 'a', epsilon, 2, ' deg ', 6, epsilon*0.02, 'less' )
NumericMultipleChoice( 'b', c, 2, ' m ', 6, c*0.08, 'both' )
NumericMultipleChoice( 'c', T, 1, ' kNm ', 1, T*0.12, 'both' )
NumericMultipleChoice( 'd', p1mp2, 2, ' Pa ', 6, p1mp2*0.18, 'greater' )
NumericMultipleChoice( 'e', p1mp2e, 1, ' Pa', 6, p1mp2e*0.15, 'greater' )
NumericMultipleChoice( 'f', Fx, 1, ' kN ', 6, Fx*0.2, 'both' )

####################################################################

[a_answer_0] (true ) 86.84  deg 
[a_answer_1] (false) 72.70  deg 
[a_answer_2] (false) 81.18  deg 
[a_answer_3] (false) 75.72  deg 
[a_answer_4] (false) 84.27  deg 
[a_answer_5] (false) 77.92  deg 

[b_answer_0] (false) 2.22  m 
[b_answer_1] (false) 1.51  m 
[b_answer_2] (false) 2.44  m 
[b_answer_3] (false) 1.78  m 
[b_answer_4] (false) 1.30  m 
[b_answer_5] (true ) 2.04  m 

[c_answer_0] (true ) 97.3  kNm 

[d_answer_0] (false) 78.71  Pa 
[d_answer_1] (true ) 35.69  Pa 
[d_answer_2] (false) 48.04  Pa 
[d_answer_3] (false) 89.59  Pa 
[d_answer_4] (false) 68.05  Pa 
[d_answer_5] (false) 60.14  Pa 

[e_answer_0] (true ) 35.8  Pa
[e_answer_1] (false) 68.6  Pa
[e_answer_2] (false) 42.4  Pa
[e_answer_3] (false) 78.4  Pa
[e_answer_4] (false) 49.1  Pa
[e_answer_5] (false) 58.9  Pa

[f_answer_0] (false) -33.3  kN 
[f_answer_1] (false) 1.3  kN 
[f_answer_2] (true ) -45.1  kN 
[f_answer_3] (false) -21.2  kN 
[f_answer_4] (false) -61.5  kN 
[f_answer_5] (false) -10.6  kN 

